In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'gpu')
jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.inpatient_interface import Patients

ModuleNotFoundError: No module named 'lib.ehr.inpatient_concepts'

In [ ]:
import logging
logging.root.level = logging.DEBUG

In [ ]:
# from lib.ehr.coding_scheme import MIMIC4Procedures, MIMIC4ProcedureGroups
# from lib.ehr.coding_scheme import MIMIC4Input, MIMIC4InputGroups

# cproc = MIMIC4Procedures()
# cproc_g = MIMIC4ProcedureGroups()
# cinp = MIMIC4Input()
# cinp_g = MIMIC4InputGroups()

In [ ]:
# Assign the folder of the dataset to `DATA_FILE`.
import dask

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR), dask.config.set(scheduler='processes'):
    m4icu_dataset = load_dataset('M4ICU')
   

In [ ]:
splits = m4icu_dataset.random_splits([0.8, 0.9], random_seed=42, balanced='subjects')

In [ ]:
preprocessing = m4icu_dataset.fit_preprocessing(splits[0])

In [ ]:
m4icu_dataset.apply_preprocessing(preprocessing)

In [ ]:
with dask.config.set(scheduler='processes', num_workers=12):
    m4inpatients = Patients(m4icu_dataset).load_subjects(num_workers=12)

In [ ]:
# m4inpatients.size_in_bytes() / 1024 ** 3

In [ ]:
# val_batch = m4inpatients.device_batch(splits[1])

In [ ]:
# tst_batch = m4inpatients.device_batch(splits[2])

In [ ]:
# val_batch.size_in_bytes() / 1024 ** 3, tst_batch.size_in_bytes() / 1024 ** 3

In [ ]:
batch = m4inpatients.device_batch(splits[0][:32])

In [ ]:
batch.size_in_bytes() / 1024 ** 3

In [ ]:
len(batch.subjects)

In [ ]:
batch.n_admissions()

In [ ]:
batch.n_segments()

In [ ]:
batch.n_obs_times()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# a = m4inpatients_jax.obs_coocurrence_matrix
# a = np.array(a)
# plt.imshow(a, cmap='hot', interpolation='nearest')
# plt.show()

In [ ]:
# s = batch.subjects[splits[0][6]].admissions[0].interventions.input_
# s

In [ ]:
# batch.interval_hours(splits[0][:10])

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [ ]:
from lib.ml.in_icenode import InICENODE, InICENODEDimensions
import jax.random as jrandom

In [ ]:
dims = InICENODEDimensions(state_m=15, 
                state_dx_e=10,
                state_obs_e=25,
                input_e=10,
                proc_e=10,
                demo_e=5,
                int_e=15)
key = jrandom.PRNGKey(0)

m = InICENODE(dims=dims, 
              scheme=m4icu_dataset.scheme,
              key=key)

In [ ]:
# res = m.batch_predict(m4inpatients.device_batch(), leave_pbar=True)

In [ ]:
from lib.ml import InTrainer, MetricsHistory
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC,
                      CodeGroupTopAlarmAccuracy, LossMetric, MetricsCollection)

from lib.ml import MinibatchLogger, EvaluationDiskWriter, ParamsDiskWriter, ConfigDiskWriter

In [ ]:
config = {        
    "batch_size": 32,
    "lr": 1e-3,
    "epochs": 150,
    "opt": "adam",
    "reg_hyperparams": None
}
trainer = InTrainer(**config)
expt_dir = 'inicenode'

In [ ]:
metrics = [
    CodeAUC(m4inpatients),
    UntilFirstCodeAUC(m4inpatients),
    AdmissionAUC(m4inpatients),
    LossMetric(m4inpatients)
]
reporters = [
        MinibatchLogger(config),
#         EvaluationDiskWriter(output_dir=expt_dir),
#         ParamsDiskWriter(output_dir=expt_dir),
#         ConfigDiskWriter(output_dir=expt_dir, config=config)
    ]
metrics = MetricsCollection(metrics)
history = MetricsHistory(metrics)

In [ ]:
    
splits = m4inpatients.random_splits([0.9, 0.95], 
                                    balanced='admissions')
res = trainer(m, m4inpatients, splits=splits, history=history, 
             reporters=reporters)

In [ ]:
# import jax.tree_util as jtu
# import jax.numpy as jnp
# import equinox as eqx

# jtu.tree_map(lambda x: f'{x.shape} {jnp.any(jnp.isnan(x)).item()}' if eqx.is_array(x) else None , m)

In [ ]:
# emb_subj = {i: m.f_emb(s) for i, s in m4inpatients.device_batch().subjects.items()}